In [8]:
#!pip3 install folium
#!pip3 install googlemaps
#!pip3 install pymongo

In [85]:
import pandas as pd
import numpy as np 
from pymongo import MongoClient
import folium
import googlemaps
import requests
import json

In [86]:
# crear base de datos
client=MongoClient()
db=client.A
colec = db.a

In [87]:
# vamos a ver todas categorías que existen en la base
query = list(colec.find({'$and':[{'number_of_employees': {'$lte':50,'$gte':10}},\
                               {'founded_year':{'$gte': 2008}},\
                                {'offices':{'$ne':True}}]},\
                      {'_id':0, 'name':1,'category_code':1,'number_of_employees':1,\
                       'founded_year':1,'offices':1}))

In [88]:
query = pd.DataFrame(query)
print(query)
query=query.dropna()
query.index = range(len(query))
len(query)

             name category_code  number_of_employees  founded_year  \
0         GoingOn      software                   40          2008   
1     Crunchyroll   games_video                   50          2008   
2        CircleUp       finance                   11          2011   
3         PeekYou        search                   20          2012   
4           GENWI        mobile                   25          2010   
..            ...           ...                  ...           ...   
581  Conversition           web                   10          2008   
582      phinditt   advertising                   10          2008   
583         Jelli    enterprise                   12          2008   
584    Tiny Speck   games_video                   45          2009   
585        Getyoo        mobile                   10          2009   

                                               offices  
0    [{'description': 'GoingOn Networks, Inc.', 'ad...  
1    [{'description': '', 'address1': '', 'ad

585

In [89]:
index_drop = [i for i in range(len(query)) if query['offices'][i] == []]
query = query.drop(index_drop) 
query.index = range(len(query))
len(index_drop)

69

In [90]:
len(query)

516

In [91]:
query.head()

,name,category_code,number_of_employees,founded_year,offices
0,GoingOn,software,40,2008,"[{'description': 'GoingOn Networks, Inc.', 'ad..."
1,Crunchyroll,games_video,50,2008,"[{'description': '', 'address1': '', 'address2..."
2,CircleUp,finance,11,2011,"[{'description': '', 'address1': '', 'address2..."
3,PeekYou,search,20,2012,"[{'description': None, 'address1': None, 'addr..."
4,GENWI,mobile,25,2010,"[{'description': '', 'address1': '4966 El Cami..."


In [92]:
query.offices[100][0]

{'description': 'Milan Office',
 'address1': 'Via Andrea Cesalpino, 60',
 'address2': '',
 'zip_code': '20128',
 'city': 'Milan',
 'state_code': None,
 'country_code': 'ITA',
 'latitude': 45.507014,
 'longitude': 9.2298577}

In [93]:
def get_first(data):
    res=[]
    ofi=[]
    
    data=data['offices']
    for e in data:
        principal=None   # solo las tienen geodata
        if e[0]['latitude'] and e[0]['longitude']:
            principal={'type':'Point',
                       'city': e[0]['city'],
                       'country code': e[0]['country_code'],
                       'coordinates':[
                           e[0]['longitude'],
                           e[0]['latitude']
                           
                       ]}
            
        ofi.append(principal)
        
        res.append({
            'total_offices':len(e),
            'lat':e[0]['latitude'],
            'lng':e[0]['longitude'],
            'oficina_principal':principal
        })
    
    return res, ofi

In [94]:

from pandas.io.json import json_normalize

first_office=json_normalize(get_first(query)[0])
first_office['oficina_principal']=get_first(query)[1]
first_office.head()

,total_offices,lat,lng,oficina_principal.type,oficina_principal.city,oficina_principal.country code,oficina_principal.coordinates,oficina_principal
0,1,37.782263,-122.392142,Point,San Francisco,USA,"[-122.392142, 37.782263]","{'type': 'Point', 'city': 'San Francisco', 'co..."
1,1,37.781265,-122.393229,Point,San Francisco,USA,"[-122.393229, 37.781265]","{'type': 'Point', 'city': 'San Francisco', 'co..."
2,1,NaN,NaN,NaN,NaN,NaN,NaN,None
3,1,40.757929,-73.985506,Point,New York,USA,"[-73.985506, 40.757929]","{'type': 'Point', 'city': 'New York', 'country..."
4,1,33.817100,-111.903500,Point,Los Altos,USA,"[-111.9035, 33.8171]","{'type': 'Point', 'city': 'Los Altos', 'countr..."


In [95]:
df_clean=pd.concat([query, first_office], axis=1)
df_clean = df_clean.drop(['offices','oficina_principal','oficina_principal.coordinates','oficina_principal.type'],axis = 1)
df_clean.head()

,name,category_code,number_of_employees,founded_year,total_offices,lat,lng,oficina_principal.city,oficina_principal.country code
0,GoingOn,software,40,2008,1,37.782263,-122.392142,San Francisco,USA
1,Crunchyroll,games_video,50,2008,1,37.781265,-122.393229,San Francisco,USA
2,CircleUp,finance,11,2011,1,NaN,NaN,NaN,NaN
3,PeekYou,search,20,2012,1,40.757929,-73.985506,New York,USA
4,GENWI,mobile,25,2010,1,33.817100,-111.903500,Los Altos,USA


In [96]:
df_clean.isnull().sum()

name                                0
category_code                       0
number_of_employees                 0
founded_year                        0
total_offices                       0
lat                               132
lng                               132
oficina_principal.city            132
oficina_principal.country code    132
dtype: int64

In [97]:
len(df_clean)

516

In [98]:
data = df_clean.dropna()
data.index = range(len(data))

In [99]:
len(data)

384

In [100]:
data.columns

Index(['name', 'category_code', 'number_of_employees', 'founded_year',
       'total_offices', 'lat', 'lng', 'oficina_principal.city',
       'oficina_principal.country code'],
      dtype='object')

In [101]:
#filtro por categorías que me interesan
data_cat = data[(data.category_code == 'analytics') |(data.category_code == 'network_hosting')|(data.category_code == 'web')]

len(data_cat)
data_cat.index = range(len(data_cat))
data_cat.head()


,name,category_code,number_of_employees,founded_year,total_offices,lat,lng,oficina_principal.city,oficina_principal.country code
0,Fixya,web,30,2013,1,37.566879,-122.323895,San Mateo,USA
1,Webnode,network_hosting,30,2008,1,49.225902,16.569282,Brno,CZE
2,First30Days,web,17,2008,1,38.257035,-85.717393,Louisville,USA
3,YouNoodle,analytics,18,2010,1,37.776659,-122.395875,San Francisco,USA
4,CityIN,web,11,2008,1,35.861660,104.195397,,CHN


In [102]:
len(data_cat)

89

In [103]:
# MAPA
mapa=folium.Map(location=[19.41, -99.16], tiles='cartodbpositron', zoom_start=1)
lat = list(data_cat['lat'])
long = list(data_cat['lng'])

for i in range(len(lat)):
    folium.Marker([lat[i],long[i]], icon=folium.Icon(color = "red")).add_to(mapa)

In [104]:
# mapa tiene 89 posibles ubicaciones, vamos a filtrar por mas cosas
# Filtramos por que tengan una sola oficina
data_cat = data_cat[(data_cat.total_offices == 1)]
len(data_cat)

75

In [105]:
data_cat.head()

,name,category_code,number_of_employees,founded_year,total_offices,lat,lng,oficina_principal.city,oficina_principal.country code
0,Fixya,web,30,2013,1,37.566879,-122.323895,San Mateo,USA
1,Webnode,network_hosting,30,2008,1,49.225902,16.569282,Brno,CZE
2,First30Days,web,17,2008,1,38.257035,-85.717393,Louisville,USA
3,YouNoodle,analytics,18,2010,1,37.776659,-122.395875,San Francisco,USA
4,CityIN,web,11,2008,1,35.861660,104.195397,,CHN


In [106]:
set(data_cat['oficina_principal.country code'])

{'AUS',
 'BEL',
 'CAN',
 'CHE',
 'CHN',
 'CZE',
 'DEU',
 'FRA',
 'GBR',
 'IND',
 'IRL',
 'NLD',
 'NOR',
 'NZL',
 'PER',
 'SGP',
 'USA'}

In [107]:
data_cat.to_csv('datos_limpios.csv')

Vamos dividir las coordenadas dependiendo de si son noroeste, noreste, sureste, suroeste. 

Latitud 

N  (+)   S (-) 


Longitud


E  (+)   0 (-) 

In [108]:
nor_este = data_cat[(data_cat.lat > 0) & (data_cat.lng > 0)].reset_index()
nor_oeste = data_cat[(data_cat.lat > 0) & (data_cat.lng < 0)].reset_index()

sur_este = data_cat[(data_cat.lat < 0) & (data_cat.lng > 0)].reset_index()
sur_oeste = data_cat[(data_cat.lat < 0) & (data_cat.lng < 0)].reset_index()

In [125]:
nor_este

,index,name,category_code,number_of_employees,founded_year,total_offices,lat,lng,oficina_principal.city,oficina_principal.country code
0,1,Webnode,network_hosting,30,2008,1,49.225902,16.569282,Brno,CZE
1,4,CityIN,web,11,2008,1,35.861660,104.195397,,CHN
2,5,BuyerRaja,web,20,2008,1,13.033353,77.578768,Bangalore,IND
3,7,Park Hill,web,10,2008,1,12.869360,74.842560,Mangalore,IND
4,17,Jibes,web,15,2008,1,52.091381,5.148893,Utrecht,NLD
5,29,IT Stocklist,web,12,2008,1,50.710267,7.066468,Bonn,DEU
6,33,Calameo,web,10,2008,1,48.871174,2.308019,Paris,FRA
7,48,JobsTAXI,web,12,2008,1,1.279684,103.842646,Singapore,SGP
8,49,TRIBAX,network_hosting,12,2008,1,52.527310,13.392249,Berlin,DEU
9,58,JobsByRef,web,10,2008,1,13.029274,80.248153,Chennai,IND


In [109]:
print(len(nor_este))
print(len(nor_oeste))
print(len(sur_este))
print(len(sur_oeste))

17
53
4
1


In [110]:
mapita =folium.Map(location=[0,0], tiles='cartodbpositron', zoom_start=2)

def mapa(db, colores):
    lat = list(db['lat'])
    long = list(db['lng'])
    
    for i in range(len(lat)):
        folium.Marker([lat[i],long[i]], icon=folium.Icon(color = str(colores))).add_to(mapita)
    
    return mapita
mapa(nor_este, colores = 'blue')
mapa(nor_oeste, colores = 'darkblue')
mapa(sur_este, colores = 'green')
mapa(sur_oeste, colores = 'darkgreen')

In [111]:
#mapita.save('mapita1.html')

In [112]:
lat = list(data_cat['lat'])
long = list(data_cat['lng'])

In [113]:
#  I. Condición: Aeropuertos cerca
req = requests.get('https://gist.githubusercontent.com/tdreyno/4278655/raw/7b0762c09b519f40397e4c3e100b097d861f5588/airports.json')
req

<Response [200]>

In [114]:
results=req.json()
airports=pd.DataFrame(results)

In [115]:
airports.head()

,code,lat,lon,name,city,state,country,woeid,tz,phone,type,email,url,runway_length,elev,icao,direct_flights,carriers
0,AAA,-17.3595,-145.494,Anaa Airport,Anaa,Tuamotu-Gambier,French Polynesia,12512819,Pacific/Midway,,Airports,,,4921,7,NTGA,2,1
1,AAE,36.8236,7.8103,El Mellah Airport,El Tarf,Annaba,Algeria,12510325,Africa/Algiers,,Airports,,,9843,16,DABB,6,2
2,AAL,57.0952,9.85606,Aalborg Airport,Norresundby,Nordjylland,Denmark,12512587,Europe/Copenhagen,,Airports,,http://www.aal.dk/,8700,10,EKYT,7,10
3,AAM,-24.8,31.5333,Mala Mala,Mala Mala,,South Africa,55921381,Africa/Johannesburg,,Airports,,,4420,1200,FAMD,1,1
4,AAN,24.25,55.75,Al Ain Airport,Ayn al Faydah,Abu Dhabi,United Arab Emirates,12523371,Asia/Dubai,,Airports,,,None,None,OMAL,12,9


In [116]:
air = airports.drop(['carriers','direct_flights','elev','email','phone','runway_length','tz','url','woeid'], axis=1)
air = air[air.type == 'Airports']
air.columns =['code', 'lat', 'lng', 'name', 'city', 'state', 'country', 'type','icao']

In [117]:
air.head()

,code,lat,lng,name,city,state,country,type,icao
0,AAA,-17.3595,-145.494,Anaa Airport,Anaa,Tuamotu-Gambier,French Polynesia,Airports,NTGA
1,AAE,36.8236,7.8103,El Mellah Airport,El Tarf,Annaba,Algeria,Airports,DABB
2,AAL,57.0952,9.85606,Aalborg Airport,Norresundby,Nordjylland,Denmark,Airports,EKYT
3,AAM,-24.8,31.5333,Mala Mala,Mala Mala,,South Africa,Airports,FAMD
4,AAN,24.25,55.75,Al Ain Airport,Ayn al Faydah,Abu Dhabi,United Arab Emirates,Airports,OMAL


In [118]:
air['lat']=air['lat'].astype(float)
air['lng']=air['lng'].astype(float)

In [119]:
Anor_este = air[(air.lat > 0) & (air.lng > 0)].reset_index()
Anor_oeste = air[(air.lat > 0) & (air.lng < 0)].reset_index()

Asur_este = air[(air.lat < 0) & (air.lng > 0)].reset_index()
Asur_oeste = air[(air.lat < 0) & (air.lng < 0)].reset_index()

In [120]:
print(len(Anor_este))
print(len(Anor_oeste))
print(len(Asur_este))
print(len(Asur_oeste))


1391
1413
450
246


1

In [74]:
'''
import math
def calculate_distance(lat1, lon1, lat2, lon2):
        '''
        * Calculates the distance between two points given their (lat, lon) co-ordinates.
        * It uses the Spherical Law Of Cosines (http://en.wikipedia.org/wiki/Spherical_law_of_cosines):
        *
        * cos(c) = cos(a) * cos(b) + sin(a) * sin(b) * cos(C)                        (1)
        *
        * In this case:
        * a = lat1 in radians, b = lat2 in radians, C = (lon2 - lon1) in radians
        * and because the latitude range is  [-π/2, π/2] instead of [0, π]
        * and the longitude range is [-π, π] instead of [0, 2π]
        * (1) transforms into:
        *
        * x = cos(c) = sin(a) * sin(b) + cos(a) * cos(b) * cos(C)
        *
        * Finally the distance is arccos(x)
        '''

        if ((lat1 == lat2) and (lon1 == lon2)):
            return 0

        try:
            delta = lon2 - lon1
            a = math.radians(lat1)
            b = math.radians(lat2)
            C = math.radians(delta)
            x = math.sin(a) * math.sin(b) + math.cos(a) * math.cos(b) * math.cos(C)
            distance = math.acos(x) # in radians
            distance  = math.degrees(distance) # in degrees
            distance  = distance * 60 # 60 nautical miles / lat degree
            distance = distance * 1852 # conversion to meters
            distance  = round(distance)
            return distance;
        except:
            return 0
            
        '''

In [ ]:
# VAMOS A INTENTAR ALGO NUEVO, PASAR LAS COORDENADASA A COORDENADAS CARTESIANAS.

In [131]:
import math

def cartesian(latitude, longitude, elevation = 0):
    # Convert to radians
    latitude = latitude * (math.pi / 180)
    longitude = longitude * (math.pi / 180)

    R = 6371 # 6378137.0 + elevation  # relative to centre of the earth
    X = R * math.cos(latitude) * math.cos(longitude)
    Y = R * math.cos(latitude) * math.sin(longitude)
    Z = R * math.sin(latitude)
    return (X, Y, Z)


In [134]:
from scipy import spatial

places = []
for index, row in nor_este.iterrows():
    coordinates = [row['lat'], row['lng']]
    cartesian_coord = cartesian(*coordinates)
    places.append(cartesian_coord)

tree = spatial.KDTree(places)


In [ ]:
def find_population(lat, lon):
    cartesian_coord = cartesian(lat, lon)
    closest = tree.query([cartesian_coord], p = 2)
    index = closest[1][0]
    return {
        'name' : geonames.name[index],
        'latitude' : geonames.latitude[index],
        'longitude' : geonames.longitude[index],
        'population' : geonames.population[index],
        'distance' : closest[0][0]
    }

In [127]:
df = pd.DataFrame([{'c1':10, 'c2':100}, {'c1':11,'c2':110}, {'c1':12,'c2':120}])

In [128]:
for index, row in df.iterrows():
    print(row['c1'], row['c2'])

10 100
11 110
12 120


In [129]:
df

,c1,c2
0,10,100
1,11,110
2,12,120
